In [6]:
import pandas as pd
import numpy as np
import json
from scipy.sparse import csr_matrix, identity
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, auc_score
from sklearn.metrics import accuracy_score, f1_score
import itertools
import csv
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA

## Load imputed data

In [7]:
df_imputed_original = pd.read_csv('../data/processed/imputed_data.csv')

## Label Construction
To train the recommendation system, we need to construct labels that indicating whether a person has purchased certain financial products, which can be derived from the characteristics of demographic data and categorical spending data.
Joint features are used for label construction, while purchase behavior (e.g., spending in categories like restaurants, utilities, or digital goods) gives insight into interests and lifestyle, it doesn't tell the full story about a customer's financial capacity, needs, or creditworthiness — all of which are critical for financial product targeting.   

Median and quantiles for certain features are computed for threshold-based rules.

In [8]:
income_median = df_imputed_original['yearly_income'].median()
num_cards_median = df_imputed_original['num_credit_cards'].median()
credit_upper = df_imputed_original['credit_score'].quantile(0.75)
income_80 = df_imputed_original['yearly_income'].quantile(0.80)  

# Define relevant spending categories and compute their 75th percentile values
spending_categories = [
    'Retail Stores', 'Restaurants & Eating Places', 'Clothing & Fashion', 
    'Movies & Theaters', 'Sports & Recreational Activities', 'Freight & Trucking', 
    'Medical & Healthcare Services', 'Postal Services - Government Only', 
    'Digital Goods & Computers', 'Telecommunications & Media', 'Utilities & Home Services', 
    'Automotive & Transportation Services', 'Steel & Metal Products', 'Machinery & Tools', 
    'Rail & Bus Transport', 'Hotels & Accommodation', 'Legal & Financial Services'
]
spending_upper = {cat: df_imputed_original[cat].quantile(0.75) for cat in spending_categories}

## Rules for label construction

In [9]:
# 1. Rewards Credit Card:
#    Conditions: (a) any spending category (Retail, Restaurants, Fashion, Movies, Sports) above its upper quantile,
#                (b) credit score above upper quantile,
#                (c) number of credit cards above or equal to median.
def label_rewards_credit_card(row):
    score = 0
    cats = ['Retail Stores', 'Restaurants & Eating Places', 'Clothing & Fashion', 
            'Movies & Theaters', 'Sports & Recreational Activities']
    if any(row[cat] >= spending_upper[cat] for cat in cats):
        score += 1
    if row['credit_score'] >= credit_upper:
        score += 1
    if row['num_credit_cards'] >= num_cards_median:
        score += 1
    return min(score, 3)

# 2. Insurance Solutions:
#    Conditions: (a) any spending category (Freight, Healthcare, Government Postal) above its upper quantile,
#                (b) credit score between median and 80th percentile,
#                (c) yearly income above or equal to median.
def label_insurance_solutions(row):
    score = 0
    cats = ['Freight & Trucking', 'Medical & Healthcare Services', 'Postal Services - Government Only']
    if any(row[cat] >= spending_upper[cat] for cat in cats):
        score += 1
    credit_median = df_imputed_original['credit_score'].quantile(0.50)
    credit_80 = df_imputed_original['credit_score'].quantile(0.80)
    if credit_median < row['credit_score'] <= credit_80:
        score += 1
    if row['yearly_income'] >= income_median:
        score += 1
    return min(score, 3)

# 3. Digital Financing:
#    Conditions: (a) any spending category above its upper quantile,
#                (b) yearly income above median,
#                (c) credit score above upper quantile.
def label_digital_financing(row):
    score = 0
    cats = ['Digital Goods & Computers', 'Telecommunications & Media']
    if any(row[cat] >= spending_upper[cat] for cat in cats):
        score += 1
    if row['yearly_income'] >= income_median:
        score += 1
    if row['credit_score'] >= credit_upper:
        score += 1
    return min(score, 3)

# 4. Home Improvement Loan (loan-related):
#    Conditions: Credit score must exceed upper quantile (else 0),
#                (a) spending in Utilities & Home Services above its upper quantile,
#                (b) yearly income above median.
def label_home_improvement_loan(row):
    score = 0 
    if row['credit_score'] >= credit_upper:
        score += 1
    if row['Utilities & Home Services'] >= spending_upper['Utilities & Home Services']:
        score += 1
    if row['yearly_income'] >= income_median:
        score += 1
    return min(score, 3)

# 5. Auto & Vehicle Financing (loan-related):
#    Conditions: Credit score must exceed upper quantile,
#                (a) spending in Automotive & Transportation Services above its upper quantile,
#                (b) yearly income above median.
def label_auto_vehicle_financing(row):
    score = 0 # Base point for meeting credit requirement
    if row['credit_score'] >= credit_upper:
        score += 1
    if row['Automotive & Transportation Services'] >= spending_upper['Automotive & Transportation Services']:
        score += 1
    if row['yearly_income'] >= income_median:
        score += 1
    return min(score, 3)

# 6. Commodity & Investment Services:
#    Conditions: (a) any spending category (Steel & Metal Products or Machinery & Tools) above its upper quantile,
#                (b) yearly income above median,
#                (c) low total debt relative to income (total_debt < 0.5*yearly_income).
def label_commodity_investment_services(row):
    score = 0
    cats = ['Steel & Metal Products', 'Machinery & Tools']
    if any(row[cat] >= spending_upper[cat] for cat in cats):
        score += 1
    if row['yearly_income'] >= income_80:
        score += 1
    if row['total_debt'] <= 0.5 * row['yearly_income']:
        score += 1
    return min(score, 3)

# 7. Travel Rewards Card:
#    Conditions: (a) any spending category (Rail & Bus Transport or Hotels & Accommodation) above its upper quantile,
#                (b) yearly income above median,
#                (c) number of credit cards above median.
def label_travel_rewards_card(row):
    score = 0
    cats = ['Rail & Bus Transport', 'Hotels & Accommodation']
    if any(row[cat] >= spending_upper[cat] for cat in cats):
        score += 1
    if row['yearly_income'] >= income_median:
        score += 1
    if row['num_credit_cards'] >= num_cards_median:
        score += 1
    return min(score, 3)

# 8. Savings/Investment Plans:
#    Conditions remain unchanged.
def label_savings_investment_plans(row):
    score = 0
    if row['Debit'] >= 1.5 * row['Credit']:
        score += 1
    if row['total_debt'] < 0.5 * row['yearly_income']:
        score += 1
    if row['retirement_age'] - row['current_age'] <= 15:
        score += 1
    return min(score, 3)

# 9. Wealth Management & Savings:
#    Conditions: (a) spending in Legal & Financial Services above its upper quantile,
#                (b) yearly income above median and low total debt,
#                (c) credit score above upper quantile.
def label_wealth_management_savings(row):
    score = 0
    if row['Legal & Financial Services'] >= spending_upper['Legal & Financial Services']:
        score += 1
    if row['yearly_income'] >= income_80 and row['total_debt'] < 0.5 * row['yearly_income']:
        score += 1
    if row['credit_score'] >= credit_upper:
        score += 1
    return min(score, 3)

## Computed labels for each financial product

In [10]:
# Apply label functions to construct new label columns.
df_imputed_original['Label_Rewards_Credit_Card'] = df_imputed_original.apply(label_rewards_credit_card, axis=1)
df_imputed_original['Label_Insurance_Solutions'] = df_imputed_original.apply(label_insurance_solutions, axis=1)
df_imputed_original['Label_Digital_Financing'] = df_imputed_original.apply(label_digital_financing, axis=1)
df_imputed_original['Label_Home_Improvement_Loan'] = df_imputed_original.apply(label_home_improvement_loan, axis=1)
df_imputed_original['Label_Auto_Vehicle_Financing'] = df_imputed_original.apply(label_auto_vehicle_financing, axis=1)
df_imputed_original['Label_Commodity_Investment_Services'] = df_imputed_original.apply(label_commodity_investment_services, axis=1)
df_imputed_original['Label_Travel_Rewards_Card'] = df_imputed_original.apply(label_travel_rewards_card, axis=1)
df_imputed_original['Label_Savings_Investment_Plans'] = df_imputed_original.apply(label_savings_investment_plans, axis=1)
df_imputed_original['Label_Wealth_Management_Savings'] = df_imputed_original.apply(label_wealth_management_savings, axis=1)

print("Sample label counts:")
df_imputed_original[['Label_Rewards_Credit_Card', 'Label_Insurance_Solutions',
                           'Label_Digital_Financing', 'Label_Home_Improvement_Loan',
                           'Label_Auto_Vehicle_Financing', 'Label_Commodity_Investment_Services',
                           'Label_Travel_Rewards_Card', 'Label_Savings_Investment_Plans',
                           'Label_Wealth_Management_Savings']].head()

Sample label counts:


,Label_Rewards_Credit_Card,Label_Insurance_Solutions,Label_Digital_Financing,Label_Home_Improvement_Loan,Label_Auto_Vehicle_Financing,Label_Commodity_Investment_Services,Label_Travel_Rewards_Card,Label_Savings_Investment_Plans,Label_Wealth_Management_Savings
0,3,3,3,3,3,2,3,1,1
1,2,1,1,1,1,1,3,1,0
2,1,1,0,0,0,0,1,1,1
3,2,0,1,0,0,0,1,1,0
4,2,2,2,1,1,1,2,1,1


## Save imputed data with label

In [11]:
df_imputed_original.to_csv('../data/processed/imputed_data_with_label.csv', index=False)

Computes the top-3 accuracy for a recommender system model.
This metric evaluates how often the true purchased items for each user appear in the model's top-3 predicted recommendations.
Users with no recorded purchases are excluded from accuracy computation.

Business Rationale:
-------------------
- Reflects realistic scenarios: In real-world recommendation systems, users interact with only a small subset of items.
- Focuses on meaningful users: Only evaluates users who made actual purchases — avoids inflating performance with inactive users.
- Top-3 is practical: Many businesses (e.g., e-commerce, fintech) care about whether their top few suggestions convert.
- Scalable scoring: Gives partial credit (e.g., 0.5) when only some of the true purchases are recommended, reflecting degrees of success.

In [12]:
def compute_top3_accuracy_for_fold(model, X_val, interactions_val, item_features, k=3):
    num_users = X_val.shape[0]
    top3_acc = []
    users_with_purchases = 0  # Track users who actually bought something
    
    for user_id in range(num_users):
        # Predict scores for all items for this user
        scores = model.predict(user_id, np.arange(interactions_val.shape[1]),
                             user_features=X_val,
                             item_features=item_features)
        
        # Get top k predicted item indices
        top3_indices = np.argsort(-scores)[:k]
        
        # Get true purchased item indices
        true_positives = set(np.where(interactions_val[user_id].toarray().flatten() == 1)[0])
        n_true = len(true_positives)
        
        # Skip users with no purchases (do not count them in accuracy)
        if n_true == 0:
            continue
        
        users_with_purchases += 1
        
        # Case 1: User purchased 3+ items
        if n_true >= 3:
            count = len(set(top3_indices).intersection(true_positives))
            top3_acc.append(count / 3.0)
        
        # Case 2: User purchased exactly 2 items
        elif n_true == 2:
            intersection = set(top3_indices).intersection(true_positives)
            if len(intersection) == 2:
                top3_acc.append(1.0)
            elif len(intersection) == 1:
                top3_acc.append(0.5)
            else:
                top3_acc.append(0)
        
        # Case 3: User purchased exactly 1 item
        elif n_true == 1:
            if top3_indices[0] in true_positives:
                top3_acc.append(1.0)
            elif len(set(top3_indices[1:]).intersection(true_positives)) > 0:
                top3_acc.append(0.5)
            else:
                top3_acc.append(0)
    
    # Return 0 if no users made purchases (edge case)
    if users_with_purchases == 0:
        return 0.0
    
    return np.mean(top3_acc)

## Define grid search function for `lightfm`, as it cannot run `GridSearchCV` function directly.

The `grid_search_cv` function performs a comprehensive grid search to optimize hyperparameters for a LightFM recommendation model using 5-fold cross-validation. It first extracts and standardizes user features based on a provided feature list, converting them into a sparse matrix, and then constructs an interaction matrix and an identity-based item features matrix. For each combination of hyperparameters (including loss function, number of components, learning rate, epochs, and regularization parameters), the function splits the training data into folds, applies upsampling on minority classes for each product column in the training interactions, and trains a LightFM model on the imputed data. It then evaluates the model on the validation set using both a standard precision@3 metric and a custom top-3 accuracy function, averaging the scores over the folds. All results are recorded and printed, and the function returns the best hyperparameters based on the highest average top-3 accuracy.

In [13]:
def grid_search_cv(feature_list, X_train_full, Y_train_bin):
    # Extract the features from X_train_full based on the provided feature list.
    X_train_features = X_train_full[feature_list].copy()
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_features)
    user_features = csr_matrix(X_train_scaled)
    
    # Build the interaction matrix from Y_train_bin.
    interactions = csr_matrix(Y_train_bin.values)
    num_items = interactions.shape[1]
    item_features = identity(num_items, format='csr')
    
    # Define hyperparameter grid.
    param_grid = {
        'loss': ['warp', 'bpr','logistic'],
        'no_components': [16, 32, 64],
        'learning_rate': [0.001, 0.01, 0.05],
        'epochs': [30, 50],
        'user_alpha': [1e-5, 1e-4],
        'item_alpha': [1e-5, 1e-4]
    }
    
    # Set up 5-fold CV.
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    grid_results = []
    upsample_factor = 2

    for loss, no_components, learning_rate, epochs, user_alpha, item_alpha in itertools.product(
        param_grid['loss'],
        param_grid['no_components'],
        param_grid['learning_rate'],
        param_grid['epochs'],
        param_grid['user_alpha'],
        param_grid['item_alpha']
    ):
        fold_top3_acc = []
        fold_prec = []

        for train_idx, val_idx in kf.split(user_features):
            X_train_cv = user_features[train_idx]
            X_val_cv = user_features[val_idx]
            
            # Get the training interactions for this fold.
            fold_train = interactions[train_idx].toarray().astype(float)
            
            # Upsample the minority class for each product column in the training fold.
            for j in range(fold_train.shape[1]):
                pos_count = np.sum(fold_train[:, j] == 1)
                neg_count = np.sum(fold_train[:, j] == 0)
                if pos_count / fold_train.shape[0] < 0.3:
                    fold_train[:, j] = np.where(fold_train[:, j] == 1,
                                                fold_train[:, j] * upsample_factor,
                                                fold_train[:, j])
                elif neg_count / fold_train.shape[0] < 0.3:
                    fold_train[:, j] = np.where(fold_train[:, j] == 0,
                                                fold_train[:, j] * upsample_factor,
                                                fold_train[:, j])
            fold_train_sparse = csr_matrix(fold_train)
            
            # Validation interactions (untouched).
            fold_val = interactions[val_idx].toarray()
            fold_val_sparse = csr_matrix(fold_val)
            
            # Train LightFM on this fold.
            model_cv = LightFM(loss=loss, no_components=no_components,
                               learning_rate=learning_rate,
                               user_alpha=user_alpha,
                               item_alpha=item_alpha,
                               random_state=42)
            model_cv.fit(fold_train_sparse,
                         user_features=X_train_cv,
                         item_features=item_features,
                         epochs=epochs,
                         num_threads=4)
            
            # Standard precision@3.
            prec = precision_at_k(model_cv, fold_val_sparse,
                                  user_features=X_val_cv,
                                  item_features=item_features,
                                  k=3).mean()
            # Compute custom top-3 accuracy using our function.
            top3_acc = compute_top3_accuracy_for_fold(model_cv, X_val_cv, fold_val_sparse, item_features, k=3)
            
            fold_top3_acc.append(top3_acc)
            fold_prec.append(prec)
        
        avg_top3_acc = np.mean(fold_top3_acc)
        avg_prec = np.mean(fold_prec)
        
        grid_results.append({
            'loss': loss,
            'no_components': no_components,
            'learning_rate': learning_rate,
            'epochs': epochs,
            'user_alpha': user_alpha,
            'item_alpha': item_alpha,
            'top3_accuracy': avg_top3_acc,
            'precision@3': avg_prec
        })
        
        print(f"Params: loss={loss}, components={no_components}, "
              f"lr={learning_rate}, epochs={epochs}, user_alpha={user_alpha}, item_alpha={item_alpha} -> "
              f"Top3 Accuracy: {avg_top3_acc:.4f}, Precision@3: {avg_prec:.4f}")
    
    best_params = max(grid_results, key=lambda x: x['top3_accuracy'])
    return best_params, grid_results

## Binarize the ordinal labels: define a "positive" interaction if label >= 2.

## Feature and Data Splitting Explanation

In the subsequent code snippet, we define three sets of columns: `target_cols` for various spending categories, `feature_cols` for customer and financial attributes used in imputation, and `label_cols` representing the labels for different financial product recommendations. The target variable `Y` is created from the imputed original dataset using the label columns, then binarized by converting values greater than or equal to 2 into 1s, indicating suitability for a particular product. Two user feature sets are prepared: a 'base' set (only using fundamental features) and an 'expanded' set (which includes additional spending categories). These feature sets are stored in a dictionary, and the full dataset is then split into training and testing subsets using a standard 80/20 split, ensuring that the test set remains unmodified for subsequent evaluation of the recommendation model.

In [14]:
target_cols = ['Automotive & Transportation Services', 'Clothing & Fashion',
       'Digital Goods & Computers', 'Electronics & Appliances',
       'Freight & Trucking', 'Hotels & Accommodation',
       'Legal & Financial Services', 'Machinery & Tools',
       'Medical & Healthcare Services', 'Movies & Theaters',
       'Postal Services - Government Only', 'Rail & Bus Transport',
       'Restaurants & Eating Places', 'Retail Stores',
       'Sports & Recreational Activities', 'Steel & Metal Products',
       'Telecommunications & Media', 'Utilities & Home Services']

# List of feature columns used for imputation
feature_cols = ['current_age', 'retirement_age', 'birth_month', 'gender',
       'latitude', 'longitude', 'yearly_income', 'total_debt', 'credit_score',
       'num_credit_cards', 'Credit', 'Debit', 'Debit (Prepaid)']

label_cols = ['Label_Rewards_Credit_Card', 'Label_Insurance_Solutions',
              'Label_Digital_Financing', 'Label_Home_Improvement_Loan',
              'Label_Auto_Vehicle_Financing', 'Label_Commodity_Investment_Services',
              'Label_Travel_Rewards_Card', 'Label_Savings_Investment_Plans',
              'Label_Wealth_Management_Savings']

Y = df_imputed_original[label_cols].copy()
Y_bin = (Y >= 2).astype(int)

# Prepare user features for tuning.
# Define two feature sets:
base_features = feature_cols.copy()
expanded_features = feature_cols + target_cols  # expanded: include spending categories

# We'll try both feature sets.
feature_sets = {
    'base': base_features,
    'expanded': expanded_features
}

# Split the data into train and test (test set remains unmodified).
X = df_imputed_original.copy()  # full data
X_train_full, X_test, Y_train_bin, Y_test_bin = train_test_split(X, Y_bin, test_size=0.2, random_state=42)


## Run both base features and expanded features in hyper-parameter tuning, find best params

In [ ]:
print("\n--- Running grid search for feature set: base ---")
best_params_base, all_results_base = grid_search_cv(base_features, X_train_full, Y_train_bin)
print(f"Best params for base features: {best_params_base}")
print("\n--- Running grid search for feature set: expanded ---")
best_params_expand, all_results_expand = grid_search_cv(expanded_features, X_train_full, Y_train_bin)
print(f"Best params for expanded features: {best_params_expand}")



--- Running grid search for feature set: base ---
Params: loss=warp, components=16, lr=0.001, epochs=30, user_alpha=1e-05, item_alpha=1e-05 -> Top3 Accuracy: 0.7454, Precision@3: 0.6211
Params: loss=warp, components=16, lr=0.001, epochs=30, user_alpha=1e-05, item_alpha=0.0001 -> Top3 Accuracy: 0.7452, Precision@3: 0.6211
Params: loss=warp, components=16, lr=0.001, epochs=30, user_alpha=0.0001, item_alpha=1e-05 -> Top3 Accuracy: 0.7452, Precision@3: 0.6211
Params: loss=warp, components=16, lr=0.001, epochs=30, user_alpha=0.0001, item_alpha=0.0001 -> Top3 Accuracy: 0.7457, Precision@3: 0.6213
Params: loss=warp, components=16, lr=0.001, epochs=50, user_alpha=1e-05, item_alpha=1e-05 -> Top3 Accuracy: 0.7446, Precision@3: 0.6208
Params: loss=warp, components=16, lr=0.001, epochs=50, user_alpha=1e-05, item_alpha=0.0001 -> Top3 Accuracy: 0.7450, Precision@3: 0.6211
Params: loss=warp, components=16, lr=0.001, epochs=50, user_alpha=0.0001, item_alpha=1e-05 -> Top3 Accuracy: 0.7447, Precision@3

In [43]:
best_params_base ={'loss': 'warp', 'no_components': 16, 'learning_rate': 0.05, 'epochs': 50, 'user_alpha': 1e-05, 'item_alpha': 1e-05}
best_params_expand = {'loss': 'warp', 'no_components': 64, 'learning_rate': 0.05, 'epochs': 50, 'user_alpha': 1e-05, 'item_alpha': 0.0001}
print(best_params_base)
print(best_params_expand)

{'loss': 'warp', 'no_components': 16, 'learning_rate': 0.05, 'epochs': 50, 'user_alpha': 1e-05, 'item_alpha': 1e-05}
{'loss': 'warp', 'no_components': 64, 'learning_rate': 0.05, 'epochs': 50, 'user_alpha': 1e-05, 'item_alpha': 0.0001}


## Define evaluation functions for test datasets

In [44]:
def evaluate_lightfm_model(X_train_full, X_test, Y_train_bin, Y_test_bin, feature_set, best_params, label_cols):
    # 1. Feature Selection and Standardization
    X_train_selected = X_train_full[feature_set].copy()
    X_test_selected = X_test[feature_set].copy()
    
    scaler_final = StandardScaler()
    X_train_final = scaler_final.fit_transform(X_train_selected)
    X_test_final = scaler_final.transform(X_test_selected)
    
    # Convert to sparse matrices for efficiency
    user_features_train_final = csr_matrix(X_train_final)
    user_features_test_final = csr_matrix(X_test_final)

    # 2. Prepare Interaction Data
    final_interactions_train = csr_matrix(Y_train_bin.values)
    final_interactions_test = csr_matrix(Y_test_bin.values)
    num_items = len(label_cols)
    final_item_features = identity(num_items, format='csr')

    # 3. Train Final Model with Best Parameters
    model_final = LightFM(
        loss=best_params['loss'],
        no_components=best_params['no_components'],
        learning_rate=best_params['learning_rate'],
        user_alpha=best_params['user_alpha'],
        item_alpha=best_params['item_alpha'],
        random_state=42
    )

    model_final.fit(
        final_interactions_train,
        user_features=user_features_train_final,
        item_features=final_item_features,
        epochs=best_params['epochs'],
        num_threads=4
    )

    # 4. Evaluation Metrics
    # Precision@3
    final_precision = precision_at_k(
        model_final,
        final_interactions_test,
        user_features=user_features_test_final,
        item_features=final_item_features,
        k=3
    ).mean()

    # Custom Top-3 Accuracy
    custom_top3_accuracy = compute_top3_accuracy_for_fold(
        model_final, 
        user_features_test_final, 
        final_interactions_test, 
        final_item_features, 
        k=3
    )

    # 5. Generate Recommendations (Key Part)
    top3_recommendations = {}
    for user_id in range(user_features_test_final.shape[0]):
        # Predict scores for all items
        scores = model_final.predict(
            user_id, 
            np.arange(num_items),
            user_features=user_features_test_final,
            item_features=final_item_features
        )
        
        # Get indices of top 3 highest scores
        top3_indices = np.argsort(-scores)[:3]
        
        # Map indices to product names
        recommended_products = [label_cols[idx] for idx in top3_indices]
        top3_recommendations[user_id] = recommended_products

    return {
        'precision@3': final_precision,
        'custom_top3_accuracy': custom_top3_accuracy
    }, top3_recommendations

In [47]:
metrics, recommendations = evaluate_lightfm_model(X_train_full, X_test, Y_train_bin, Y_test_bin, 
                                                    base_features, best_params_base, label_cols)
print("\nFinal Model Metrics (using base features):")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")
print("\nTop 3 product recommendations for sample test users:")
for uid in list(recommendations.keys())[:5]:
    print(f"User {uid}: {recommendations[uid]}")


Final Model Metrics (using base features):
precision@3: 0.6877
custom_top3_accuracy: 0.8228

Top 3 product recommendations for sample test users:
User 0: ['Label_Rewards_Credit_Card', 'Label_Travel_Rewards_Card', 'Label_Digital_Financing']
User 1: ['Label_Rewards_Credit_Card', 'Label_Travel_Rewards_Card', 'Label_Insurance_Solutions']
User 2: ['Label_Travel_Rewards_Card', 'Label_Rewards_Credit_Card', 'Label_Digital_Financing']
User 3: ['Label_Travel_Rewards_Card', 'Label_Rewards_Credit_Card', 'Label_Insurance_Solutions']
User 4: ['Label_Digital_Financing', 'Label_Auto_Vehicle_Financing', 'Label_Home_Improvement_Loan']


In [48]:
metrics, recommendations = evaluate_lightfm_model(X_train_full, X_test, Y_train_bin, Y_test_bin, 
                                                    expanded_features, best_params_expand, label_cols)
print("\nFinal Model Metrics (using expanded features):")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")
print("\nTop 3 product recommendations for sample test users:")
for uid in list(recommendations.keys())[:5]:
    print(f"User {uid}: {recommendations[uid]}")



Final Model Metrics (using expanded features):
precision@3: 0.7367
custom_top3_accuracy: 0.8844

Top 3 product recommendations for sample test users:
User 0: ['Label_Travel_Rewards_Card', 'Label_Rewards_Credit_Card', 'Label_Insurance_Solutions']
User 1: ['Label_Travel_Rewards_Card', 'Label_Rewards_Credit_Card', 'Label_Insurance_Solutions']
User 2: ['Label_Rewards_Credit_Card', 'Label_Travel_Rewards_Card', 'Label_Home_Improvement_Loan']
User 3: ['Label_Travel_Rewards_Card', 'Label_Commodity_Investment_Services', 'Label_Rewards_Credit_Card']
User 4: ['Label_Auto_Vehicle_Financing', 'Label_Digital_Financing', 'Label_Home_Improvement_Loan']


## Conclusion: expanded features shows a higher accuracy and precision for the top 3 predictions, as the spending data provides a more hollistic view of the customer's behaviour which could greatly benefits the prediction results.

In [ ]:
# Convert dict to DataFrame
cleaned_dict = {
    user_id: [label.replace('Label_', '') for label in labels]
    for user_id, labels in recommendations.items()
}

# Convert to DataFrame
df = pd.DataFrame.from_dict(cleaned_dict, orient='index')
df.columns = ['Top1', 'Top2', 'Top3']
# Save to CSV
df.to_csv('recommendations.csv', index_label='UserID')